<a href="https://colab.research.google.com/github/lucacerab/DSEthesis-transformers/blob/main/BERTopic_ECB_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Topic Modeling on ECB speeches dataset using BERTopic

In [ ]:
# after installing it restart the notebook
%%capture
!pip install bertopic

##Data preparation

In [ ]:
import pandas as pd

ecb = pd.read_csv('https://www.ecb.europa.eu/press/key/shared/data/all_ECB_speeches.csv?48aec91d00ba1a66db570a6d4d163845', sep="|", encoding='utf8').dropna().reset_index(drop=True)
ecb.head()

,date,speakers,title,subtitle,contents
0,2021-07-14,Isabel Schnabel,A new strategy for a changing world,"Speech by Isabel Schnabel, Member of the Execu...",SPEECH A new strategy for a changing world...
1,2021-07-11,Christine Lagarde,"Climate Change and Central Banks: Analysing, A...","Speech by Christine Lagarde, President of the ...",SPEECH Climate Change and Central Banks: A...
2,2021-07-08,"Christine Lagarde,Luis de Guindos",Opening remarks to the press conference (with ...,"Christine Lagarde, President of the ECB, Luis ...",OPENING REMARKS PRESS CONFERENCE Christi...
3,2021-07-03,Isabel Schnabel,Escaping low inflation?,"Speech by Isabel Schnabel, Member of the Execu...",SPEECH Escaping low inflation? Speech by...
4,2021-06-29,Christine Lagarde,Financing a green and digital recovery,"Speech by Christine Lagarde, President of the ...",SPEECH Financing a green and digital recov...


In [ ]:
ecb.shape

(2473, 5)

In [ ]:
print(ecb.speakers.nunique(), 'speakers:', ecb.speakers.unique())

28 speakers: ['Isabel Schnabel' 'Christine Lagarde' 'Christine Lagarde,Luis de Guindos'
 'Luis de Guindos' 'Fabio Panetta' 'Frank Elderson' 'Philip R. Lane'
 'Yves Mersch' 'Benoît Cœuré' 'Sabine Lautenschläger' 'Mario Draghi'
 'Peter Praet' 'Vítor Constâncio' 'Jörg Asmussen'
 'José Manuel González-Páramo' 'Lorenzo Bini Smaghi' 'Jürgen Stark'
 'Jean-Claude Trichet' 'Gertrude Tumpel-Gugerell' 'Lucas Papademos'
 'Otmar Issing' 'Tommaso Padoa-Schioppa' 'Eugenio Domingo Solans'
 'Willem F. Duisenberg' 'Sirkka Hämäläinen' 'Christian Noyer'
 'Willem F. Duisenberg,Eugenio Domingo Solans' 'Alexandre Lamfalussy']


In [ ]:
ecb.contents[0]

'   SPEECH  A new strategy for a changing world   Speech by Isabel Schnabel, Member of the Executive Board of the ECB, at the virtual Financial Statements series hosted by the Peterson Institute for International Economics Frankfurt am Main, 14 July 2021 Last week the Governing Council of the European Central Bank (ECB) published its new monetary policy strategy.[1] It was the ECB’s first review of its strategy since 2003, with most features of the framework still dating back to its founding years. Since those times, the world economy has changed in fundamental ways. About half of today’s ten largest global firms by stock market capitalisation did not exist when the euro was launched in 1999. In that year, imports accounted for around 30% of euro area economic activity; on the eve of the pandemic, this share had increased to 45%. And whereas in 2000 there were on average 24 people aged 65 and over for every 100 persons of working age in the euro area, this ratio stood at 32 last year. 

In [ ]:
# filter 'SPEECH' from contents column
import re

ecb.contents = ecb.apply(lambda row: re.sub(r"SPEECH", "", row.contents).strip(), 1)

In [ ]:
ecb.contents[0]

'A new strategy for a changing world   Speech by Isabel Schnabel, Member of the Executive Board of the ECB, at the virtual Financial Statements series hosted by the Peterson Institute for International Economics Frankfurt am Main, 14 July 2021 Last week the Governing Council of the European Central Bank (ECB) published its new monetary policy strategy.[1] It was the ECB’s first review of its strategy since 2003, with most features of the framework still dating back to its founding years. Since those times, the world economy has changed in fundamental ways. About half of today’s ten largest global firms by stock market capitalisation did not exist when the euro was launched in 1999. In that year, imports accounted for around 30% of euro area economic activity; on the eve of the pandemic, this share had increased to 45%. And whereas in 2000 there were on average 24 people aged 65 and over for every 100 persons of working age in the euro area, this ratio stood at 32 last year. These chang

In [ ]:
# keep only english speeches
%%capture
!pip install langdetect
from langdetect import detect

def detect_en(text):
    try:
        return detect(text) == 'en'
    except:
        return False

In [ ]:
df = ecb[ecb['contents'].apply(detect_en)].reset_index(drop=True)
df.head()

,date,speakers,title,subtitle,contents
0,2021-07-14,Isabel Schnabel,A new strategy for a changing world,"Speech by Isabel Schnabel, Member of the Execu...",A new strategy for a changing world Speech b...
1,2021-07-11,Christine Lagarde,"Climate Change and Central Banks: Analysing, A...","Speech by Christine Lagarde, President of the ...","Climate Change and Central Banks: Analysing, A..."
2,2021-07-08,"Christine Lagarde,Luis de Guindos",Opening remarks to the press conference (with ...,"Christine Lagarde, President of the ECB, Luis ...",OPENING REMARKS PRESS CONFERENCE Christine ...
3,2021-07-03,Isabel Schnabel,Escaping low inflation?,"Speech by Isabel Schnabel, Member of the Execu...",Escaping low inflation? Speech by Isabel Sch...
4,2021-06-29,Christine Lagarde,Financing a green and digital recovery,"Speech by Christine Lagarde, President of the ...",Financing a green and digital recovery Speec...


In [ ]:
df.shape

(2314, 5)

In [ ]:
time = df.date.to_list()
speeches = df.contents.to_list()

## Topic Modeling

In [ ]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer

#ngram_range=(1, 2), vectorizer_model=vectorizer_model
#vectorizer_model = CountVectorizer(stop_words="english")
topic_model = BERTopic(language='english', nr_topics='auto', verbose=True)

In [ ]:
topics, _ = topic_model.fit_transform(speeches)

Batches:   0%|          | 0/73 [00:00<?, ?it/s]

2021-08-19 09:09:23,338 - BERTopic - Transformed documents to Embeddings
2021-08-19 09:09:36,471 - BERTopic - Reduced dimensionality with UMAP
2021-08-19 09:09:36,611 - BERTopic - Clustered UMAP embeddings with HDBSCAN
2021-08-19 09:10:03,509 - BERTopic - Reduced number of topics from 42 to 26


In [ ]:
freq = topic_model.get_topic_info()
print(len(freq), 'topics found')

26 topics found


In [ ]:
freq.head(10)

,Topic,Count,Name
0,-1,987,-1_financial_euro_policy_with
1,0,214,0_policy_monetary_inflation_rates
2,1,159,1_euro_currency_monetary_international
3,2,139,2_european_we_euro_economic
4,3,124,3_financial_crisis_banks_market
5,4,106,4_financial_integration_market_markets
6,5,91,5_inflation_growth_euro_economic
7,6,47,6_payments_payment_digital_services
8,7,44,7_we_duisenberg_as_euro
9,8,43,8_banking_banks_supervision_supervisory


In [ ]:
topic_model.get_topic(2)

[('european', 0.013006621915188204),
 ('we', 0.011446293621680098),
 ('euro', 0.010514502650533466),
 ('economic', 0.010505588746366246),
 ('europe', 0.010298147060237146),
 ('monetary', 0.009801458600746469),
 ('at', 0.009742332716138818),
 ('policy', 0.009330155252962135),
 ('central', 0.008765851454780848),
 ('our', 0.008498030384443129)]

In [ ]:
topic_model.get_topic(5)

[('inflation', 0.019876302037333038),
 ('growth', 0.01599521257949989),
 ('euro', 0.013037371030419213),
 ('economic', 0.011790536166280239),
 ('at', 0.011514893007851188),
 ('monetary', 0.011404332789565352),
 ('policy', 0.010511886805650912),
 ('stability', 0.008519813027432764),
 ('over', 0.008063084378150336),
 ('financial', 0.007917040876201225)]

In [ ]:
topic_model.get_topic(7)

[('we', 0.022005046648586747),
 ('duisenberg', 0.015422498579115877),
 ('as', 0.013453463340419448),
 ('euro', 0.012530686835903032),
 ('would', 0.011953397061262756),
 ('at', 0.010681509066488322),
 ('think', 0.010347226325109175),
 ('growth', 0.010148849194736857),
 ('monetary', 0.009967444825097692),
 ('ecb', 0.009718037323195499)]

In [ ]:
topics_over_time = topic_model.topics_over_time(docs=speeches, 
                                                topics=topics, 
                                                timestamps=time, 
                                                global_tuning=True, 
                                                evolution_tuning=True, 
                                                nr_bins=20)

20it [00:26,  1.31s/it]


In [ ]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=26)

In [ ]:
# selecting most interesting topics
fig = topic_model.visualize_topics_over_time(topics_over_time, topics=[0,1,3,4,6,8,10,11])
fig

In [ ]:
import plotly.express as px

fig.write_html("/content/DTM.html")

In [ ]:
# extract 3 most representative docs for a specific topic (monetary policy)
docs = topic_model.get_representative_docs(topic=0)
docs

['Hearing at the Committee on Economic and Monetary Affairs of the European Parliament   Introductory statement by Mario Draghi, President of the ECB, Brussels, 8 July 2013 Madam Chair, Honourable members of the Committee on Economic and Monetary Affairs, It is a great pleasure for me to be back with your committee for our last exchange of views before the summer break. At this time, it is worth taking stock of progress over the past 12 months. Clearly, financial conditions in the euro area today are more stable and resilient than they were last summer. This is partly due to our determined monetary policy actions. Governments and parliaments have also played a key role in the relative return of confidence and stability by undertaking courageous reforms, at both the national and European levels.  Yet despite this progress, the euro area still faces considerable challenges. The economy is still weak. Financial fragmentation remains. This challenges the very concept of the Single Market. 

In [ ]:
with open('docs_monetarypolicy.txt', 'w') as f:
    for i in docs:
        f.write("%s\n" % i)

In [ ]:
topic_model.visualize_barchart(top_n_topics=6, height=700)

In [ ]:
topic_model.save("BERTopic_ECB")